In [ ]:
import fitz
import re
import os
import pickle
import json

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    total_pages = len(doc)
    
    if total_pages <= 20:
        print(f"Skipping {pdf_path} as it has less than 20 pages.")
        return ""
    
    for page_num in range(10, total_pages - 10):
        text += doc[page_num].get_text("text") + "\n"
    return text

def clean_and_chunk_text(text, chunk_size=500):
    text = re.sub(r'\s+', ' ', text) 
    sentences = re.split(r'(?<=[.!?]) +', text)  
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + " "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "
    if chunk:
        chunks.append(chunk.strip())
    return chunks

pdf_folder = "../public/resources"
pdf_files = os.listdir(pdf_folder)
pdf_files = [pdf for pdf in pdf_files if pdf.endswith(".pdf")][:30]
print(f"Total PDFs: {len(pdf_files)}")

all_chunks = []
pdf_texts = []

for pdf in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf)
    raw_text = extract_text_from_pdf(pdf_path)
    if raw_text:
        chunks = clean_and_chunk_text(raw_text)
        all_chunks.extend(chunks)
        pdf_texts.append({"title": os.path.splitext(pdf)[0], "filename": pdf})

print(f"Total Chunks Created: {len(all_chunks)}")
print(all_chunks[:3])

with open("all_chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

print(f"✅ Extracted and saved {len(all_chunks)} chunks.")


Total PDFs: 30
Total Chunks Created: 37768
['5 1.5.2 Central Processing Unit CPU is the major component which interprets and executes software instructions. It also control the operation of all other components such as memory, input and output units. It accepts binary data as input, process the data according to the instructions and provide the result as output. The CPU has three components which are Control unit, Arithmetic and logic unit (ALU) and Memory unit.', '1.5.2.1 Arithmetic and Logic Unit The ALU is a part of the CPU where various computing functions are performed on data. The ALU performs arithmetic operations such as addition, subtraction, multiplication, division and logical operations. The result of an operation is stored in internal memory of CPU. The logical operations of ALU promote the decision-making ability of a computer. 1.5.2.2 Control Unit The control unit controls the flow of data between the CPU, memory and I/O devices.', 'It also controls the entire operation 

In [3]:
import logging
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Loading Sentence Transformer model on MPS...")
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="mps")

logging.info("Encoding chunks with optimized batch size (256)...")
embeddings = model.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True, batch_size=256)

embeddings = np.ascontiguousarray(embeddings.astype(np.float32))
logging.info(f"Encoding complete! Total vectors: {len(embeddings)}, Dimension: {embeddings.shape[1]}")

dimension = embeddings.shape[1]
n_clusters = 1024 

logging.info(f"Initializing FAISS IVF index with {n_clusters} clusters...")
quantizer = faiss.IndexFlatL2(dimension) 
index = faiss.IndexIVFFlat(quantizer, dimension, n_clusters, faiss.METRIC_L2)


logging.info("Training FAISS index...")
index.train(embeddings)

logging.info("Adding embeddings to FAISS index...")
index.add(embeddings)

faiss.write_index(index, "faiss_index.bin")
logging.info("FAISS index stored successfully at 'faiss_index.bin'.")

print(f"Total vectors: {len(embeddings)}")


/Users/rahulmarban/Documents/Coding/kalvikaram/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-08 18:07:00,299 - INFO - Loading Sentence Transformer model on MPS...
2025-03-08 18:07:00,301 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
2025-03-08 18:07:06,489 - INFO - Encoding chunks with optimized batch size (256)...
Batches: 100%|██████████| 148/148 [08:27<00:00,  3.43s/it]
2025-03-08 18:15:34,469 - INFO - Encoding complete! Total vectors: 37768, Dimension: 384
2025-03-08 18:15:34,472 - INFO - Initializing FAISS IVF index with 1024 clusters...
2025-03-08 18:15:34,480 - INFO - Training FAISS index...
WARNING clustering 37768 points to 1024 centroids: please provide at least 39936 training points
2025-03-08 18:15:35,384 - INFO - Adding embeddings to FAI

Total vectors: 37768


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np
from dotenv import load_dotenv
import os
import google.generativeai as genai

with open("all_chunks.pkl", "rb") as f:
    all_chunks = pickle.load(f)

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
index = faiss.read_index("faiss_index.bin")

load_dotenv()
genai.configure(api_key=os.getenv("GENAI_API_KEY"))
generative_model = genai.GenerativeModel("gemini-2.0-flash")

def search_faiss(query, top_k=5, distance_threshold=0.5):
    """
    Retrieve top matching text chunks from FAISS index with a distance filter.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = [
        all_chunks[idx] 
        for idx, dist in zip(indices[0], distances[0]) 
        if idx < len(all_chunks) and dist < distance_threshold
    ]
    return results if results else ["No relevant information found."]

def generate_answer(query):
    retrieved_chunks = search_faiss(query)
    context = "\n".join(retrieved_chunks)
    prompt = f"""
You are an tamil AI assistant that uses textbook content to answer student queries with tamil books.
Given the query and context below, generate a relevant answer based on the context, if the answer is not in the context, you can generate the answer based on the context but dont mention that its not in context and dont say sorry.
Make it simple and easy to understand and learn, if its a english name like a theory name then maintain english but if there is a tamil meaning for it then explain in tamil.
Dont literally translate to tamil, say it in tamil meaning instead of literal english to tamil. You can give summarization about it and talk about stuff thats around the topic that was asked.
Im studying in tamil medium so please make it easy to understand and learn and also if it has a complex english word then mention the english part in side with paranthesis. Use proper tamil words and translation

If i ask something thats wayyy off topic then just say that has nothing to do with the topic.

Context:
{context}

Question: {query}
Answer:"""
    response = generative_model.generate_content(prompt)
    return response.text.strip()

2025-03-08 18:15:36,203 - INFO - Use pytorch device_name: mps
2025-03-08 18:15:36,203 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2


In [5]:
queries = [
    "What is gibbs free energy",
    "Difference between mass and weight",
    "What is the first law of thermodynamics",
]

for query in queries:
    response = generate_answer(query)
    print(f"\n🔍 Query: {query}")
    print(f"🤖 AI Answer: {response}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]



🔍 Query: What is gibbs free energy
🤖 AI Answer: உங்க கேள்வி Gibbs Free Energy பத்தினது. இது வேதியியல்ல (chemistry) ரொம்ப முக்கியமான ஒரு கருத்து. ஒரு வேதிவினை (chemical reaction) தானா நடக்குமா, நடக்காதான்னு கண்டுபிடிக்க இது உதவும்.

Gibbs Free Energy-ங்கிறது ஒரு சிஸ்டத்தோட (system) எனர்ஜிய குறிக்குது. ஒரு சிஸ்டம் எவ்வளவு வேலை செய்ய முடியும் அப்படிங்குறதையும் இது சொல்லும். இதை G அப்படிங்குற எழுத்தால குறிப்பாங்க.

ஒரு வேதிவினை நடக்கும்போது, Gibbs Free Energy குறையும். அதாவது, G-யோட மதிப்பு குறைஞ்சா, அந்த வினை தானா நடக்கும். G-யோட மதிப்பு அதிகமா இருந்தா, அந்த வினை தானா நடக்காது. நாம எனர்ஜி கொடுத்தா தான் நடக்கும்.

சுருக்கமா சொல்லணும்னா, Gibbs Free Energy ஒரு வேதிவினை தானா நடக்குமா, நடக்காதான்னு சொல்லுற ஒரு கருவி மாதிரி.


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]



🔍 Query: Difference between mass and weight
🤖 AI Answer: நிறை (Mass) மற்றும் எடை (Weight) ஆகியவற்றுக்கு இடையேயான வேறுபாடு என்னவென்று பார்ப்போம்.

**நிறை (Mass):**

*   நிறை என்பது ஒரு பொருளில் உள்ள பருப்பொருளின் அளவைக் குறிக்கிறது. அதாவது, ஒரு பொருள் எவ்வளவு "stuff" ஆல் ஆனது என்பதைப் பற்றியது.
*   இது ஒரு மாறாத மதிப்பு. அதாவது, நீங்கள் பூமியில் இருந்தாலும் சரி, சந்திரனில் இருந்தாலும் சரி, ஒரு பொருளின் நிறை எப்போதும் ஒரே மாதிரியாக இருக்கும்.
*   நிறையை கிலோகிராமில் (kilogram - kg) அளவிடுகிறோம்.

**எடை (Weight):**

*   எடை என்பது ஒரு பொருளின் மீது புவியீர்ப்பு விசை (gravitational force) எவ்வளவு செயல்படுகிறது என்பதைக் குறிக்கிறது.
*   எடை மாறும் மதிப்பு. ஏனெனில், புவியீர்ப்பு விசை இடத்திற்கு இடம் மாறுபடும். உதாரணமாக, பூமியில் ஒரு பொருளின் எடை அதிகமாக இருக்கும், ஆனால் சந்திரனில் குறைவாக இருக்கும்.
*   எடையை நியூட்டனில் (Newton - N) அளவிடுகிறோம்.

**சுருக்கமாக:**

நிறை என்பது ஒரு பொருளில் உள்ள பருப்பொருளின் அளவு, எடை என்பது புவியீர்ப்பு விசையால் அந்தப் பொருளின் மீது ஏற்படும் விசை.

எளிமையா

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.59s/it]



🔍 Query: What is the first law of thermodynamics
🤖 AI Answer: வெப்ப இயக்கவியலின் முதல் விதி (First law of thermodynamics) என்பது ஆற்றல் அழிவின்மை விதியினை (law of conservation of energy) அடிப்படையாகக் கொண்டது. இந்த விதியின்படி, ஒரு தனிமைப்படுத்தப்பட்ட அமைப்பில் (isolated system) ஆற்றலை உருவாக்கவோ அழிக்கவோ முடியாது, ஆனால் ஒரு வடிவத்திலிருந்து மற்றொரு வடிவத்திற்கு மாற்ற முடியும்.

எளிமையாக சொல்லனும்னா, ஒரு மூடிய அமைப்புல (closed system) இருக்கிற மொத்த ஆற்றல் எப்பவும் மாறாம இருக்கும். ஆற்றலை ஒரு இடத்துல இருந்து இன்னொரு இடத்துக்கு மாற்றலாம், இல்லனா ஒரு வகையான ஆற்றலை இன்னொரு வகையான ஆற்றலா மாற்றலாம், ஆனா அந்த மொத்த ஆற்றலோட அளவு மாறவே மாறாது.

உதாரணத்துக்கு, ஒரு வண்டிக்கு பெட்ரோல் (petrol) போடும்போது, பெட்ரோல்ல இருக்கிற இரசாயன ஆற்றல் (chemical energy) வண்டி ஓடும்போது இயக்க ஆற்றலா (kinetic energy) மாறுது. ஆனா, அந்த மொத்த ஆற்றலோட அளவு மாறாது. இதுதான் வெப்ப இயக்கவியலின் முதல் விதி சொல்லுது.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import faiss

index = faiss.read_index("faiss_index.bin")
dimension = 384

num_vectors = index.ntotal
if num_vectors < 5:
    exit()

if not hasattr(index, 'direct_map') or index.direct_map.type == 0:
    exit()

embeddings = np.zeros((num_vectors, dimension), dtype=np.float32)
for i in range(num_vectors):
    index.reconstruct(i, embeddings[i])

scaler = StandardScaler()
embeddings = scaler.fit_transform(embeddings)

perplexity_value = min(30, num_vectors - 1)
tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(8, 6))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.6)
plt.title("t-SNE Visualization of Text Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True)
plt.show()

In [ ]:
import os
import pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

file_path = "all_chunks.pkl"
with open(file_path, "rb") as f:
    all_chunks = pickle.load(f)

index = faiss.read_index("faiss_index.bin")

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

bm25 = BM25Okapi([chunk.split() for chunk in all_chunks])

query = "What is mass?"
query_embedding = model.encode([query], convert_to_numpy=True)

distances, indices = index.search(query_embedding, 5)
retrieved_chunks = [all_chunks[i] for i in indices[0] if i < len(all_chunks)]

bm25_scores = bm25.get_scores(query.split())
bm25_indices = np.argsort(bm25_scores)[::-1][:5]
bm25_chunks = [all_chunks[i] for i in bm25_indices]

# Print results
print("\nFAISS Retrieved Chunks:")
for chunk in retrieved_chunks:
    print(f"- {chunk[:200]}...")

print("\nBM25 Retrieved Chunks:")
for chunk in bm25_chunks:
    print(f"- {chunk[:200]}...")
    
print("FAISS Retrieved Indices:", indices)
print("FAISS Retrieved Distances:", distances)




FAISS Retrieved Chunks:
- UNIT-1(XI-Physics_Vol-1).indd 16 UNIT-1(XI-Physics_Vol-1).indd 16 05-01-2022 22:11:42 05-01-2022 22:11:42 www.tntextbooks.in Unit 1 Nature of Physical World and Measurement 17 1.5.2 Measurement of mas...
- The masses of objects which we shall study in this course vary over a wide range. These may vary from a tiny mass of electron (9.11×10−31kg) to the huge mass of the known universe (=1055 kg). The orde...
- Gram equivalent mass is the mass of an element (compound/ ion) that combines or displaces 1.008 g hydrogen, 8 g oxygen or 35.5 g chlorine. Elemental analysis of a compound gives the mass percentage of...
- Solution: Mass m1 = 60 kg Mass m2 = 30 kg V cms V cm s 1 1 2 1 40 30 Solution: v m m m m u m m m u v m m m m u 1 1 2 1 2 1 2 1 2 2 2 2 1 1 2 2 2 1 2 1 2m1 m m u Substituting the values, we get, v1 60 ...
- = 4 34 0 31 14 . . = ∴ Empirical formula = Na2 S H20 O14 n molar mass calculated empirical formula mass = = = 322 322 1 Na S H O 2 20 14 2 23 1 32 20